In [1]:
import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import time
from tkinter import TclError
import serial
import wave

# to display in separate Tk window
%matplotlib tk

# constants
CHUNK = 1024*5  # samples per frame
FORMAT = pyaudio.paInt16  # audio format (bytes per sample?)
CHANNELS = 1  # single channel for microphone
RATE = 44100  # samples per second

wav_file = wave.open(r"C:\Users\rajka\Downloads\Aaya Na Tu - Arjun Kanungo.wav", 'rb')

# Set up serial communication with Arduino
arduino = serial.Serial('COM4', 9600)
time.sleep(2)  # Wait for Arduino to initialize

# create matplotlib figure and axes
fig, ax = plt.subplots(figsize=(15, 7))

# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=p.get_format_from_width(wav_file.getsampwidth()),
    channels=wav_file.getnchannels(),
    rate=wav_file.getframerate(),
    input=False,
    output=True,
    frames_per_buffer=CHUNK
)

# variable for plotting
xf = np.linspace(0, RATE, CHUNK)  # frequencies (spectrum)

# create semilogx line for spectrum
line_fft, = ax.semilogx(xf, np.random.rand(CHUNK), '-', lw=2)

# format spectrum axes
ax.set_xlim(20, RATE / 2)
ax.set_xlabel('Frequency')
ax.set_ylabel('Amplitude')
ax.set_title('Spectrum')

print('Playback started')
chosen_frequencies = [9000]
chosen_indices = [np.abs(xf - freq).argmin() for freq in chosen_frequencies]

# for measuring frame rate
frame_count = 0
start_time = time.time()

# Read the audio data from the .wav file
data = wav_file.readframes(CHUNK)

while len(data) > 0:
    # convert data to integers, make np array, then offset it by 127
    data_int = np.array(struct.unpack(f"{2 * CHUNK}h", data), dtype=np.int16) + 128
    # compute FFT and update line
    yf = fft(data_int)
    line_fft.set_ydata(np.abs(yf[0:CHUNK]) / (128 * CHUNK))

    stream.write(data)

    for freq, index in zip(chosen_frequencies, chosen_indices):
        value = np.abs(yf[index]) / (128 * CHUNK)
        print(f"Frequency: {freq} Hz, Value: {value}")

        message = f"{freq}:{value}\n".encode()  # Encode message as bytes
        arduino.write(message)

    # update figure canvas
    fig.canvas.draw()
    fig.canvas.flush_events()

    # Read serial output from Arduino
    while arduino.in_waiting:
        # Read a line from the serial port
        line = arduino.readline().decode().strip()

        # Print the serial output
        print(line)

    # Read the next chunk of audio data from the .wav file
    data = wav_file.readframes(CHUNK)

print('Playback finished')


Playback started
Frequency: 9000 Hz, Value: 0.003009757876449124
Frequency: 500 Hz, Value: 1.9463824911680312
Frequency: 300 Hz, Value: 2.4956257428814306
Execution Time: 0 ms
Execution Time: 0 ms
Execution Time: 0 ms
Frequency: 9000 Hz, Value: 0.0018305959237832296
Frequency: 500 Hz, Value: 6.16634449156476
Frequency: 300 Hz, Value: 2.866719851124488
Execution Time: 0 ms
Execution Time: 0 ms
Execution Time: 0 ms
Frequency: 9000 Hz, Value: 0.008040403377927557
Frequency: 500 Hz, Value: 10.625227736473459
Frequency: 300 Hz, Value: 0.8856270678551164
Execution Time: 0 ms
Execution Time: 0 ms
Execution Time: 0 ms
Frequency: 9000 Hz, Value: 0.005177917097388041
Frequency: 500 Hz, Value: 4.935724761221248
Frequency: 300 Hz, Value: 1.192883690026448
Execution Time: 0 ms
Execution Time: 0 ms
Execution Time: 0 ms
Frequency: 9000 Hz, Value: 0.0023603108119473327
Frequency: 500 Hz, Value: 2.061180844410986
Frequency: 300 Hz, Value: 0.5993306617812932
Execution Time: 0 ms
Execution Time: 0 ms
Exe

KeyboardInterrupt: 